In [1]:
#Импорт библиотек
import telebot
from telebot import types
from imageai.Detection import ObjectDetection
import cv2
import os

In [2]:
# Инициализация бота с использованием токена API
API_TOKEN = 'Your token'
bot = telebot.TeleBot(API_TOKEN)
# Инициализация детектора объектов
detector = ObjectDetection()
model_path = "yolov3.pt"  # Путь к модели YOLOv3
detector.setModelTypeAsYOLOv3()  # Установка типа модели
detector.setModelPath(model_path)  # Установка пути к модели
detector.loadModel()  # Загрузка модели
# Словарь для цветов рамок участников дорожного движения
colors = {
    'person': (0, 255, 0),      # Зеленый для людей
    'bicycle': (255, 0, 0),     # Синий для велосипедов
    'car': (0, 0, 255),         # Красный для автомобилей
    'motorbike': (255, 255, 0), # Циан для мотоциклов
    'bus': (255, 0, 255),       # Магента для автобусов
    'truck': (0, 255, 255)      # Желтый для грузовиков
}
def detect_traffic_participants(image_path):
    """Определяет участников дорожного движения на фотографии."""
    detections = detector.detectObjectsFromImage(
        input_image=image_path,
        output_image_path=None,
        minimum_percentage_probability=30  # Минимальный порог вероятности для обнаружения
    )
    traffic_participants = ['person', 'bicycle', 'car', 'motorbike', 'bus', 'truck']
    filtered_detections = [detection for detection in detections if detection['name'] in traffic_participants]
    return filtered_detections
def draw_filtered_detections(image_path, detections):
    """Рисует рамки только вокруг участников дорожного движения."""
    image = cv2.imread(image_path)  # Чтение изображения
    for detection in detections:
        box_points = detection['box_points']  # Координаты рамки
        label = detection['name']  # Название объекта
        color = colors.get(label, (255, 255, 255))  # Цвет рамки по умолчанию белый
        # Рисование рамки вокруг обнаруженного объекта
        cv2.rectangle(image,
                      (int(box_points[0]), int(box_points[1])),
                      (int(box_points[2]), int(box_points[3])),
                      color, 2)
        # Подпись объекта с вероятностью
        cv2.putText(image,
                    f"{label} : {detection['percentage_probability']:.2f}%",
                    (int(box_points[0]), int(box_points[1] - 10)),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, color, 2)
    return image
@bot.message_handler(content_types=['photo'])
def handle_photo(message):
    # Получаем файл изображения, отправленного пользователем
    file_info = bot.get_file(message.photo[-1].file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    
    # Сохраняем файл на диск
    input_image_path = "input_image.jpg"
    with open(input_image_path, 'wb') as new_file:
        new_file.write(downloaded_file)
    
    # Обрабатываем изображение
    detections = detect_traffic_participants(input_image_path)
    detected_image = draw_filtered_detections(input_image_path, detections)
    
    # Сохраняем обработанное изображение
    output_image_path = "output_image.jpg"
    cv2.imwrite(output_image_path, detected_image)
    
    # Отправляем обратно пользователю
    with open(output_image_path, 'rb') as output_file:
        bot.send_photo(message.chat.id, output_file)
    
    # Удаляем временные файлы
    os.remove(input_image_path)
    os.remove(output_image_path)
@bot.message_handler(commands=['start'])
def send_welcome(message):
    # Приветственное сообщение при старте
    welcome_text = (
        "Добро пожаловать в нашего бота для обнаружения участников дорожного движения! 🚦\n\n"
        "Я помогу вам выявить участников дорожного движения на фотографиях и выделить их цветными рамками.\n"
        "Чтобы узнать, как использовать бота, введите команду /help."
    )
    bot.send_message(message.chat.id, welcome_text)
@bot.message_handler(commands=['help'])
def send_help(message):
    # Инструкция по использованию бота
    help_text = (
        "Привет! Я бот для обнаружения участников дорожного движения на фотографиях.\n\n"
        "Вот как вы можете меня использовать:\n"
        "- Отправьте мне фотографию с участниками дорожного движения (люди, велосипеды, автомобили, мотоциклы, автобусы, грузовики).\n"
        "- Я обработаю изображение и выделю участников цветными рамками.\n"
        "- Вы получите обратно обработанное изображение.\n\n"
        "Команды:\n"
        "/start - Начать работу с ботом.\n"
        "/help - Получить информацию о командах бота.\n"
    )
    bot.send_message(message.chat.id, help_text)
# Запуск бота
bot.polling(none_stop=True)